In [8]:
### polyphase channelizer
import numpy as np
from scipy.signal import firwin, lfilter, freqz
from scipy.signal import butter, filtfilt, resample
from scipy.fft import fft, ifft, fftshift

import plotly.graph_objects as go
import plotly.express as px

In [9]:
x_f_1 = np.random.randn(1000) + 0j
x_f_1[100:900] *= 1e-6
x_f_1_original = x_f_1.copy()
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)

x_f_2 = np.random.randn(1000) + 0j
x_f_2[200:800] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_f_2_original = x_f_2.copy()
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_2))/4000)

x_f_3 = np.random.randn(1000) + 0j
x_f_3[300:700] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_f_3_original = x_f_3.copy()
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_3))/4000)

x_f_4 = np.random.randn(1000) + 0j
x_f_4[400:600] *= 1e-6
x_t_4 = ifft(x_f_4)*len(x_f_4)
x_f_4_original = x_f_4.copy()
x_t_4 = resample(x_t_4, 4000)
x_t_4 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_4))/4000)

x_f_1 = fft(x_t_1)/len(x_t_1)
x_f_2 = fft(x_t_2)/len(x_t_1)
x_f_3 = fft(x_t_3)/len(x_t_1)
x_f_4 = fft(x_t_4)/len(x_t_1)

x_f_total = x_f_1 + x_f_2 + x_f_3 + x_f_4

x_t_total = ifft(x_f_total)*len(x_f_total)


In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Combined Wide-band Signal', line=dict(dash="dot")))    
fig.update_layout(width=700,height=400, title='Multi-Channel  Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [11]:
fig.write_html("wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Conventional Approach

In [12]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

num_taps = 64  # Number of filter taps (coefficients) - typically odd for linear phase

filter_taps = signal.firwin(num_taps, 0.25, pass_zero=True)

filter_freq_response = fft(filter_taps, 4000)

In [13]:
fig_filter = go.Figure()
fig_filter.add_trace(go.Scatter(y=filter_taps, mode='lines+markers', name=f'64-Tap Low-pass Filter'))   
fig_filter.update_layout(width=700,height=400, title='64-Tap Low-pass Filter', xaxis_title='Time Sample', yaxis_title='Magnitude (Linear)',
                  )

In [14]:
fig_filter.write_html("lpf_taps.html",
                        include_plotlyjs='cdn',
                        full_html=False)

In [15]:
### Apply frequency shift on channel 3
x_f_total_shifted = np.roll(x_f_total, -2000)
x_f_1_shifted = np.roll(x_f_1, -2000)
x_f_2_shifted = np.roll(x_f_2, -2000)
x_f_3_shifted = np.roll(x_f_3, -2000)
x_f_4_shifted = np.roll(x_f_4, -2000)

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_shifted)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_shifted)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_shifted)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_shifted)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_shifted)), mode='lines', name=f'Wide-band Signal (Frequency-Shifted)', line=dict(dash="dot")))   
fig.update_layout(width=700,height=400, title='Frequency-Shifted Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [17]:
fig.write_html("shifted_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [18]:
# 4. Apply the filter to the signal
x_f_total_filtered = filter_freq_response*x_f_total_shifted
x_f_1_filtered = filter_freq_response*x_f_1_shifted
x_f_2_filtered = filter_freq_response*x_f_2_shifted
x_f_3_filtered = filter_freq_response*x_f_3_shifted
x_f_4_filtered = filter_freq_response*x_f_4_shifted

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_filtered)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_filtered)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_filtered)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_filtered)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered)), mode='lines', name=f'Wide-band Signal (Filtered)', line=dict(dash="dot")))   
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(filter_freq_response)), mode='lines', name=f'Filter Response', line=dict(dash="dot")))   
fig.update_layout(width=700,height=400, title='Filtered Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [20]:
fig.write_html("filtered_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [21]:
x_t_total_filtered = ifft(x_f_total_filtered)*len(x_f_total_filtered)
x_t_total_filtered_downsamp = x_t_total_filtered[::4]
x_f_total_filtered_downsamp = fft(x_t_total_filtered_downsamp)/len(x_t_total_filtered_downsamp)

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered_downsamp)), mode='lines', name=f'Signal 3 (Extracted)'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_original)), mode='lines', name=f'Signal 3 (Original)'))    
fig.update_layout(width=700,height=400, title='Signal 3 Extracted by Decimation Filtering', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 1000], yaxis_range=[-150, 10])

In [23]:
fig.write_html("decim_filtered_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Polyphase Channelizer

In [206]:
def plot_complex_sepectrum(spectrum,
                           freq=None, 
                           name=f'Spectrum', 
                           fig=None, 
                           add_axis=False, 
                           plot_title=None, 
                           aspect_ratio=None, 
                           IQ_limit=1.5, 
                           scatter_mode='markers+lines',
                           marker_size=2
                           ):
    if fig is None:
        fig = go.Figure()

    if freq is None:
        freq = np.arange(len(spectrum))
    fig.add_trace(go.Scatter3d(x=freq,
                                y=np.imag(spectrum),
                                z=np.real(spectrum),
                                mode=scatter_mode, name=name,
                                marker=dict(size=marker_size),
                                opacity=1))   
    

    fig.update_layout(width=700,height=650,
                        scene=dict(
                            xaxis=dict( title="Frequency"),
                            yaxis=dict(range=[-1, 1], title="Q"),
                            zaxis=dict(range=[-1, 1], title="I"),
                            aspectmode="manual",           # allow manual control
                            aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
                            camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75),
                                        up=dict(x=0, y=0, z=1),
                                        center=dict(x=0, y=0, z=-0.2),),  # adjust the camera so labels are visible,
                        ),
                        legend=dict(
                                    orientation="h",          # horizontal
                                    yanchor="bottom",         # anchor legend to bottom
                                    y=0,                   # push legend below plot
                                    xanchor="center",         # anchor centered horizontally
                                    x=0.5
                                    ),
                        margin=dict(l=00, r=00, t=50, b=0)
                        )
    
    if aspect_ratio is not None:
        fig.update_layout(scene=dict(aspectmode="manual",           # allow manual control
                                     aspectratio=dict(x=aspect_ratio[0], 
                                                      y=aspect_ratio[1], 
                                                      z=aspect_ratio[2]),  # scale of each axis
                                     ))
    if plot_title is not None:
        fig.update_layout(title=plot_title)

    if IQ_limit is not None:
        fig.update_layout(scene=dict(yaxis=dict(range=[-IQ_limit, IQ_limit], title="Q"),
                                     zaxis=dict(range=[-IQ_limit, IQ_limit], title="I"),
                                     ))
    # plot axes
    if add_axis==True:
        fig.add_trace(go.Scatter3d(
            x=[0,len(spectrum)], y=[0,0], z=[0,0],
            mode="lines", line=dict(color="black", width=1),
            name="X=0 axis", showlegend=False
        ))

        fig.add_trace(go.Scatter3d(
            x=[0,0], y=[-IQ_limit,IQ_limit], z=[0,0],
            mode="lines", line=dict(color="black", width=1),
            name="X=0 axis", showlegend=False
        ))

        fig.add_trace(go.Scatter3d(
            x=[0,0], y=[0,0], z=[-IQ_limit,IQ_limit],
            mode="lines", line=dict(color="black", width=1),
            name="X=0 axis", showlegend=False
        ))

        fig.add_trace(go.Scatter3d(
            x=[len(spectrum),len(spectrum)], y=[-IQ_limit,IQ_limit], z=[0,0],
            mode="lines", line=dict(color="black", width=1),
            name="X=0 axis", showlegend=False
        ))

        fig.add_trace(go.Scatter3d(
            x=[len(spectrum),len(spectrum)], y=[0,0], z=[-IQ_limit,IQ_limit],
            mode="lines", line=dict(color="black", width=1),
            name="X=0 axis", showlegend=False
        ))

    return fig

In [85]:
def create_slider_fig(figs_list, labels_list, combined_plot_title="Combined Figure"):
    combined = go.Figure()

    # Add the first figure's traces as the initial data
    combined.add_traces(figs_list[0].data)

    # Build frames from all figs
    frames = []
    for i, f in enumerate(figs_list, start=1):
        frames.append(go.Frame(data=f.data, name=f"frame{i}"))

    combined.frames = frames

    # Add slider
    combined.update_layout(
        sliders=[{
            "steps": [
                {"args": [[f.name], {"frame": {"duration": 0}, "mode": "immediate"}],
                "label": labels_list[i], 
                "method": "animate"}
                for i, f in enumerate(combined.frames)
            ],
            "transition": {"duration": 0},
            "x": 0.1, "y": -0.1, "len": 0.9
        }],
        width=figs_list[0].layout['width'],
        height=figs_list[0].layout['height']+150,
        scene=figs_list[0].layout['scene'],
        title=combined_plot_title,
        legend=dict(
            orientation="h",          # horizontal
            yanchor="bottom",         # anchor legend to bottom
            y=0,                   # push legend below plot
            xanchor="center",         # anchor centered horizontally
            x=0.5
        )
        )
    return combined

# Test Polyphase Filter

In [196]:
num_taps = 64  # Number of filter taps (coefficients) - typically odd for linear phase
filter_taps = signal.firwin(num_taps, 0.25, pass_zero=True)

### Construct Ideal Filter
filter_freq_response = 0.00*np.ones(4000) + 0j
filter_freq_response[0:450] += 0.9
filter_freq_response[3650:] += 0.9
filter_taps = np.fft.ifft(filter_freq_response)*len(filter_freq_response)

In [211]:
fig_filter_freq = plot_complex_sepectrum(filter_freq_response, 
                                         name='Ideal Low-pass Filter Frequency Response', 
                                         plot_title='Ideal Low-pass Filter Frequency Response', 
                                         scatter_mode='markers+lines',
                                         add_axis=True,
                                         IQ_limit=1.25,
                                         marker_size=2)
fig_filter_freq.update_layout(width=800, height=600,
                          scene=dict(
                            aspectmode="manual",           # allow manual control
                            aspectratio=dict(x=1.5, y=0.5, z=0.5),  # scale of each axis
                            camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.55),
                                        up=dict(x=0, y=0, z=1),
                                        center=dict(x=0, y=0, z=-0.2),),  # adjust the camera so labels are visible,
                        ))

In [212]:
fig_filter_freq.write_html("ideal_lpf_spectrum.html",
                           include_plotlyjs='cdn',
                            full_html=False)

In [ ]:
down_samp_mask = np.zeros(4000)
down_samp_mask[0::4] = 1


filter_f_0 = fft(np.roll(filter_taps,0)*down_samp_mask, 4000)/len(filter_taps)
filter_f_1 = fft(np.roll(filter_taps,1)*down_samp_mask, 4000)/len(filter_taps)
filter_f_2 = fft(np.roll(filter_taps,2)*down_samp_mask, 4000)/len(filter_taps)
filter_f_3 = fft(np.roll(filter_taps,3)*down_samp_mask, 4000)/len(filter_taps)

filter_f_0 = fft(signal.decimate(np.roll(filter_taps,0)*down_samp_mask,4, ftype='fir'), 1000)/1000*4
filter_f_1 = fft(signal.decimate(np.roll(filter_taps,1)*down_samp_mask,4, ftype='fir'), 1000)/1000*4
filter_f_2 = fft(signal.decimate(np.roll(filter_taps,2)*down_samp_mask,4, ftype='fir'), 1000)/1000*4
filter_f_3 = fft(signal.decimate(np.roll(filter_taps,3)*down_samp_mask,4, ftype='fir'), 1000)/1000*4

In [195]:
fig_filter_downsamp = plot_complex_sepectrum(filter_f_0, name='h0', add_axis=True, IQ_limit=1.25, plot_title="Frequency Response of Down-sampled Filters")
fig_filter_downsamp = plot_complex_sepectrum(filter_f_1, name='h1', add_axis=True, IQ_limit=1.25, plot_title="Frequency Response of Down-sampled Filters", fig = fig_filter_downsamp)
fig_filter_downsamp = plot_complex_sepectrum(filter_f_2, name='h2', add_axis=True, IQ_limit=1.25, plot_title="Frequency Response of Down-sampled Filters", fig = fig_filter_downsamp)
fig_filter_downsamp = plot_complex_sepectrum(filter_f_3, name='h3', add_axis=True, IQ_limit=1.25, plot_title="Frequency Response of Down-sampled Filters", fig = fig_filter_downsamp)

fig_filter_downsamp.update_layout(width=800, height=600,
                          scene=dict(
                            aspectmode="manual",           # allow manual control
                            aspectratio=dict(x=1.5, y=0.5, z=0.5),  # scale of each axis
                            camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.55),
                                        up=dict(x=0, y=0, z=1),
                                        center=dict(x=0, y=0, z=-0.2),),  # adjust the camera so labels are visible,
                        ))
fig_filter_downsamp.show()

In [159]:
fig_filter_downsamp.write_html("downsampled_filter_spectrum.html",
                                include_plotlyjs='cdn',
                                full_html=False)

In [214]:
x_f_0 = np.ones(1000) + 0j
x_f_0[100:900] *= 1e-6
x_t_0 = ifft(x_f_0)*len(x_f_0)
x_t_0 = resample(x_t_0, 4000)
x_f_0 = fft(x_t_0)/len(x_t_0)

x_f_1 = np.ones(1000) + 0j
x_f_1[200:800] *= 1e-6
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)
x_t_1 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_1))/4000)
x_f_1 = fft(x_t_1)/len(x_t_1)

x_f_2 = np.ones(1000) + 0j
x_f_2[300:700] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_2))/4000)
x_f_2 = fft(x_t_2)/len(x_t_2)

x_f_3 = np.ones(1000) + 0j
x_f_3[400:600] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_3))/4000)
x_f_3 = fft(x_t_3)/len(x_t_3)

x_f_total = x_f_0 + x_f_1 + x_f_2 + x_f_3
x_t_total = ifft(x_f_total)*len(x_f_total)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_0)), mode='lines', name=f'ch 0 component'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'ch 1 component'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'ch 2 component'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'ch 3 component'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Combined Wide-band Signal', line=dict(dash="dot")))    
fig.update_layout(width=700,height=400, title='Multi-Channel Input Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])



In [ ]:
fig.write_html("wideband_signal_polyphase_input.html",
               include_plotlyjs='cdn',
               full_html=False)

In [ ]:
fig_stream0 = plot_complex_sepectrum(x_f_1, name='ch 0 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum")
fig_stream0 = plot_complex_sepectrum(x_f_1, name='ch 1 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
fig_stream0 = plot_complex_sepectrum(x_f_1, name='ch 2 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
fig_stream0 = plot_complex_sepectrum(x_f_1, name='ch 3 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
# fig_stream0.show()

In [161]:
### Check filtering with decimation
y_filt_f = x_f_total*fft(filter_taps, n=len(x_f_total))
x_filt_t = ifft(y_filt_f)*len(y_filt_f)
x_filt_downsamp_t = x_filt_t[0::4]
x_filt_downsamp_f = fft(x_filt_downsamp_t)/len(x_filt_downsamp_t)


In [ ]:
x_t_total_delay0_downsamp = x_t_total[0::4]
x_t_0_delay0_downsamp = x_t_0[0::4]
x_t_1_delay0_downsamp = x_t_1[0::4]
x_t_2_delay0_downsamp = x_t_2[0::4]
x_t_3_delay0_downsamp = x_t_3[0::4]

x_f_total_delay0_downsamp = fft(x_t_total_delay0_downsamp)/len(x_t_total_delay0_downsamp)
x_f_0_delay0_downsamp = fft(x_t_0_delay0_downsamp)/len(x_t_0_delay0_downsamp)
x_f_1_delay0_downsamp = fft(x_t_1_delay0_downsamp)/len(x_t_1_delay0_downsamp)
x_f_2_delay0_downsamp = fft(x_t_2_delay0_downsamp)/len(x_t_2_delay0_downsamp)
x_f_3_delay0_downsamp = fft(x_t_3_delay0_downsamp)/len(x_t_3_delay0_downsamp)


x_t_total_delay1_downsamp = x_t_total[1::4]
x_t_0_delay1_downsamp = x_t_0[1::4]
x_t_1_delay1_downsamp = x_t_1[1::4]
x_t_2_delay1_downsamp = x_t_2[1::4]
x_t_3_delay1_downsamp = x_t_3[1::4]
x_f_total_delay1_downsamp = fft(x_t_total_delay1_downsamp)/len(x_t_total_delay1_downsamp)
x_f_0_delay1_downsamp = fft(x_t_0_delay1_downsamp)/len(x_t_0_delay1_downsamp)
x_f_1_delay1_downsamp = fft(x_t_1_delay1_downsamp)/len(x_t_1_delay1_downsamp)
x_f_2_delay1_downsamp = fft(x_t_2_delay1_downsamp)/len(x_t_2_delay1_downsamp)
x_f_3_delay1_downsamp = fft(x_t_3_delay1_downsamp)/len(x_t_3_delay1_downsamp)


x_t_total_delay2_downsamp = x_t_total[2::4]
x_t_0_delay2_downsamp = x_t_0[2::4]
x_t_1_delay2_downsamp = x_t_1[2::4]
x_t_2_delay2_downsamp = x_t_2[2::4]
x_t_3_delay2_downsamp = x_t_3[2::4]
x_f_total_delay2_downsamp = fft(x_t_total_delay2_downsamp)/len(x_t_total_delay2_downsamp)
x_f_0_delay2_downsamp = fft(x_t_0_delay2_downsamp)/len(x_t_0_delay2_downsamp)
x_f_1_delay2_downsamp = fft(x_t_1_delay2_downsamp)/len(x_t_1_delay2_downsamp)
x_f_2_delay2_downsamp = fft(x_t_2_delay2_downsamp)/len(x_t_2_delay2_downsamp)
x_f_3_delay2_downsamp = fft(x_t_3_delay2_downsamp)/len(x_t_3_delay2_downsamp)


x_t_total_delay3_downsamp = x_t_total[3::4]
x_t_0_delay3_downsamp = x_t_0[3::4]
x_t_1_delay3_downsamp = x_t_1[3::4]
x_t_2_delay3_downsamp = x_t_2[3::4]
x_t_3_delay3_downsamp = x_t_3[3::4]
x_f_total_delay3_downsamp = fft(x_t_total_delay3_downsamp)/len(x_t_total_delay3_downsamp)
x_f_0_delay3_downsamp = fft(x_t_0_delay3_downsamp)/len(x_t_0_delay3_downsamp)
x_f_1_delay3_downsamp = fft(x_t_1_delay3_downsamp)/len(x_t_1_delay3_downsamp)
x_f_2_delay3_downsamp = fft(x_t_2_delay3_downsamp)/len(x_t_2_delay3_downsamp)
x_f_3_delay3_downsamp = fft(x_t_3_delay3_downsamp)/len(x_t_3_delay3_downsamp)

In [192]:
fig_stream0 = plot_complex_sepectrum(x_f_0_delay0_downsamp, name='ch 0 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum")
fig_stream0 = plot_complex_sepectrum(x_f_1_delay0_downsamp, name='ch 1 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
fig_stream0 = plot_complex_sepectrum(x_f_2_delay0_downsamp, name='ch 2 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
fig_stream0 = plot_complex_sepectrum(x_f_3_delay0_downsamp, name='ch 3 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 0 (delay=0) Spectrum", fig=fig_stream0)
# fig_stream0.show()

fig_stream1 = plot_complex_sepectrum(x_f_0_delay1_downsamp, name='ch 0 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 1 (delay=1) Spectrum")
fig_stream1 = plot_complex_sepectrum(x_f_1_delay1_downsamp, name='ch 1 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 1 (delay=1) Spectrum", fig=fig_stream1)
fig_stream1 = plot_complex_sepectrum(x_f_2_delay1_downsamp, name='ch 2 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 1 (delay=1) Spectrum", fig=fig_stream1)
fig_stream1 = plot_complex_sepectrum(x_f_3_delay1_downsamp, name='ch 3 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 1 (delay=1) Spectrum", fig=fig_stream1)
# fig_stream1.show()

fig_stream2 = plot_complex_sepectrum(x_f_0_delay2_downsamp, name='ch 0 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 2 (delay=2) Spectrum")
fig_stream2 = plot_complex_sepectrum(x_f_1_delay2_downsamp, name='ch 1 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 2 (delay=2) Spectrum", fig=fig_stream2)
fig_stream2 = plot_complex_sepectrum(x_f_2_delay2_downsamp, name='ch 2 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 2 (delay=2) Spectrum", fig=fig_stream2)
fig_stream2 = plot_complex_sepectrum(x_f_3_delay2_downsamp, name='ch 3 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 2 (delay=2) Spectrum", fig=fig_stream2)
# fig_stream2.show()

fig_stream3 = plot_complex_sepectrum(x_f_0_delay3_downsamp, name='ch 0 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 3 (delay=3) Spectrum")
fig_stream3 = plot_complex_sepectrum(x_f_1_delay3_downsamp, name='ch 1 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 3 (delay=3) Spectrum", fig=fig_stream3)
fig_stream3 = plot_complex_sepectrum(x_f_2_delay3_downsamp, name='ch 2 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 3 (delay=3) Spectrum", fig=fig_stream3)
fig_stream3 = plot_complex_sepectrum(x_f_3_delay3_downsamp, name='ch 3 component', add_axis=True, IQ_limit=1.25, plot_title="Down-sampled Stream 3 (delay=3) Spectrum", fig=fig_stream3)
# fig_stream3.show()

fig_streams = create_slider_fig([fig_stream0, fig_stream1, fig_stream2, fig_stream3],
                                ['Delay=0', 'Delay=1', 'Delay=2', 'Delay=3'],
                                combined_plot_title="Complex Spectra of Down-sampled Input Streams")

fig_streams.update_layout(width=800, height=600,
                          scene=dict(
                            aspectmode="manual",           # allow manual control
                            aspectratio=dict(x=1.75, y=0.5, z=0.5),  # scale of each axis
                            camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.55),
                                        up=dict(x=0, y=0, z=1),
                                        center=dict(x=0, y=0, z=-0.2),),  # adjust the camera so labels are visible,
                        ))  # adjust the camera so labels are visible

In [193]:
fig_streams.write_html("downsampled_streams_spectrum.html",
                       include_plotlyjs='cdn',
                          full_html=False)

In [168]:
filter_0_t = filter_taps[0::4]
filter_1_t = filter_taps[1::4]
filter_2_t = filter_taps[2::4]
filter_3_t = filter_taps[3::4]

filter_0_f = fft(filter_0_t,1000)
filter_1_f = fft(filter_1_t,1000)
filter_2_f = fft(filter_2_t,1000)
filter_3_f = fft(filter_3_t,1000)

In [169]:
y0 = ifft(fft(x_t_total_delay0_downsamp)*filter_0_f)
y0_ch0 = ifft(fft(x_t_0_delay0_downsamp)*filter_0_f)
y0_ch1 = ifft(fft(x_t_1_delay0_downsamp)*filter_0_f)
y0_ch2 = ifft(fft(x_t_2_delay0_downsamp)*filter_0_f)
y0_ch3 = ifft(fft(x_t_3_delay0_downsamp)*filter_0_f)

y1 = ifft(fft(x_t_total_delay3_downsamp)*filter_1_f)
y1_ch0 = ifft(fft(x_t_0_delay3_downsamp)*filter_1_f)
y1_ch1 = ifft(fft(x_t_1_delay3_downsamp)*filter_1_f)
y1_ch2 = ifft(fft(x_t_2_delay3_downsamp)*filter_1_f)
y1_ch3 = ifft(fft(x_t_3_delay3_downsamp)*filter_1_f)

y2 = ifft(fft(x_t_total_delay2_downsamp)*filter_2_f)
y2_ch0 = ifft(fft(x_t_0_delay2_downsamp)*filter_2_f)
y2_ch1 = ifft(fft(x_t_1_delay2_downsamp)*filter_2_f)
y2_ch2 = ifft(fft(x_t_2_delay2_downsamp)*filter_2_f)
y2_ch3 = ifft(fft(x_t_3_delay2_downsamp)*filter_2_f)

y3 = ifft(fft(x_t_total_delay1_downsamp)*filter_3_f)
y3_ch0 = ifft(fft(x_t_0_delay1_downsamp)*filter_3_f)
y3_ch1 = ifft(fft(x_t_1_delay1_downsamp)*filter_3_f)
y3_ch2 = ifft(fft(x_t_2_delay1_downsamp)*filter_3_f)
y3_ch3 = ifft(fft(x_t_3_delay1_downsamp)*filter_3_f)


y_out_t_ch0 = np.roll(y0, 0) + np.roll(y1, 1)  + np.roll(y2, 1)  + np.roll(y3, 1)
y_out_t_ch1 = np.roll(y0, 0) + 1j*np.roll(y1, 1)  + -1*np.roll(y2, 1)  + -1j*np.roll(y3, 1)
y_out_t_ch2 = np.roll(y0, 0) + -1*np.roll(y1, 1)  + np.roll(y2, 1)  + -1*np.roll(y3, 1)
y_out_t_ch3 = np.roll(y0, 0) + -1j*np.roll(y1, 1)  + -1*np.roll(y2, 1)  + 1j*np.roll(y3, 1)

y_out_f_ch0 = fft(y_out_t_ch0)/len(y_out_t_ch0)
y_out_f_ch1 = fft(y_out_t_ch1)/len(y_out_t_ch1)
y_out_f_ch2 = fft(y_out_t_ch2)/len(y_out_t_ch2)
y_out_f_ch3 = fft(y_out_t_ch3)/len(y_out_t_ch3)

In [167]:
px.line(y=20*np.log10(np.abs(y_out_f_ch0)))

In [ ]:
### each of the 4 channels sees 4 aliased copies of the filter
### so there are 16 combinations
### however, for each chanel, only 1 

In [ ]:
### ch1 sees 4 copies of the filter (due to aliasing)
### the only part that adds in phase is the one where it would have see anyway during decimation filtering.
### all other parts are out of phase and cancel out.

### same argument applies to ch2 and ch3

### for ch0, the part in the passband adds in phase, so it has the full filter response.
### all other parts are out of phase and cancel out. 

# Scratch 

In [ ]:
x = np.random.randn(12)
h = np.random.randn(6)

y_filt_down = ifft(fft(x,12)*fft(h,12))
y_filt_down = y_filt_down[0::3]

h0 = h[0::3]
h1 = h[1::3]
h2 = h[2::3]

x0 = x[0::3]
x1 = x[1::3]
x2 = x[2::3]

y0 = ifft(fft(x0,4)*fft(h0,4))
y1 = ifft(fft(x2,4)*fft(h1,4))
y2 = ifft(fft(x1,4)*fft(h2,4))

y_filt_poly = np.roll(y0, 0) + np.roll(y1, 1)  + np.roll(y2, 1)